## Challenge code
 
 In the code below, you are given a few functions:
 
 - `hydrogen_hamiltonian`: This function will return the qubit Hamiltonian of the hydrogen molecule, H$_2$, given the `coordinates` of both hydrogen atoms and the net molecular `charge`. You'll usually find H$_2$ with a charge of 0, but here we'll spice it up with a non-zero charge!
 - `num_electrons`: In subsequent functions, we'll need the total number of electrons in the hydrogen molecule we're looking at. With a charge of 0, H$_2$ usually has just 2 electrons, one per hydrogen atom. Given the `charge`, how many electrons should H$_2$ have? ***You must complete this function.***
 - `hf`: The "HF" stands for Hartree–Fock. This function's purpose is calculate the HF approximation — treat every electron as independent, electrons move under a Coulomb potential from the positively charged nuclei, and there's a mean field from the other electrons — for the ground state of the hydrogen molecule we're interested in. We'll use this later, so ***you must complete this function.***
 - `run_VQE`: This function takes the `coordinates`, `charge`, generates the HF state, defines a `cost` function and minimizes it. ***You must complete this function*** by:
   + defining the gates within the `cost` function, using the [`qml.AllSinglesDoubles`](https://docs.pennylane.ai/en/stable/code/api/pennylane.AllSinglesDoubles.html) template with `singles` and `doubles` arguments defined below; and
   + returning what we want to minimize, namely the expectation value of the hydrogen Hamiltonian! 
 
 Here are some helpful resources:
 
 - [Building molecular Hamiltonians](https://pennylane.ai/qml/demos/tutorial_quantum_chemistry.html)
 - [A brief overview of VQE](https://pennylane.ai/qml/demos/tutorial_vqe)
 - [Variational Quantum Eigensolver](https://youtu.be/4Xnxa6tzPeA)
 - [Quantum Chemistry documentation](https://docs.pennylane.ai/en/stable/introduction/chemistry.html)
 
 ### Input 

 ![img](images/daily5.png)

 ![img](images/daily5_2.png)
 
 As input to this problem, you are given:
 
 - `coordinates` (`list(float)`): the $x$, $y$, and $z$ coordinates of each hydrogen atom
 - `charge` (`int`): the charge of the hydrogen molecule. It could be positive, negative, or zero!
 
 ### Output
 
 This code must output the ground state `energy` (`float`) of the hydrogen molecule in question.
 
 If your solution matches the correct one within the given tolerance specified in `check` (in this case it's a `1e-3` relative error tolerance), the output will be `"Correct!"` Otherwise, you will receive a `"Wrong answer"` prompt.
 
 Good luck!
 ### Imports
 The cell below specifies the libraries you should use in this challenge. Run the cell to import the libraries. ***Do not modify the cell.***

In [46]:
import json
import pennylane as qml
from pennylane import numpy as np

### Code
 Complete the code below. Note that during QHack, some sections were not editable. We've marked those sections accordingly here, but you can still edit them if you wish.

In [47]:
def hydrogen_hamiltonian(coordinates, charge):
    """Calculates the qubit Hamiltonian of the hydrogen molecule.
    
    Args:
        coordinates (list(float)): Cartesian coordinates of each hydrogen molecule.
        charge (int): The electric charge given to the hydrogen molecule.

    Returns:
        (qml.Hamiltonian): A PennyLane Hamiltonian.
    """
    # qchem: Quantum Chemistry module
    # qml.qchem.molecular_hamiltonian: compute the Hamiltonian directly
    #     returns : H, qubit [we take just the hamiltonian]
    # ["H", "H"]: H_2 molecule
    # basis="STO-3G": minimal basis, consists of orthogonal orbitals
    symbols = ["H", "H"]
    # return qml.qchem.molecular_hamiltonian(symbols, coordinates, charge=charge, basis="STO-3G")[0]
    multiplicity = 1 if charge == 0 else 2
    mol = qml.qchem.Molecule(symbols, coordinates, charge=charge, basis_name="STO-3G", mult=multiplicity)
    return qml.qchem.molecular_hamiltonian(mol, method='openfermion')[0]

coords = np.array([[0.0, 0.0, -0.8], [0.0, 0.0, 0.8]])
hydrogen_hamiltonian(coords, -1)

(
    -0.21359179057257 * I(0)
  + 0.15603499574064658 * Z(0)
  + 0.15603499574064664 * Z(1)
  + -0.17937702800901079 * Z(2)
  + -0.17937702800901079 * Z(3)
  + 0.16362674468200183 * (Z(0) @ Z(1))
  + 0.04684119793391368 * (Y(0) @ X(1) @ X(2) @ Y(3))
  + -0.04684119793391368 * (Y(0) @ Y(1) @ X(2) @ X(3))
  + -0.04684119793391368 * (X(0) @ X(1) @ Y(2) @ Y(3))
  + 0.04684119793391368 * (X(0) @ Y(1) @ Y(2) @ X(3))
  + 0.11457952447635825 * (Z(0) @ Z(2))
  + 0.1614207224102719 * (Z(0) @ Z(3))
  + 0.1614207224102719 * (Z(1) @ Z(2))
  + 0.11457952447635825 * (Z(1) @ Z(3))
  + 0.16964861667411466 * (Z(2) @ Z(3))
)

We get the hamiltonian as sum of pauli strings. an hamiltonian we can use in a quantum circuit

In [48]:
def num_electrons(charge):
    """The total number of electrons in the hydrogen molecule.
    
    Args:
        charge (int): The electric charge given to the hydrogen molecule.

    Returns: 
        (int): The number of electrons.
    """
    # End of uneditable section #

    # Put your solution here #
    # neutral H2 molecule has 2 electrons
    # H2+ (+1 positive charge) has 1 electron
    # H2- (-1 negative charge) has 3 electrons
    return -charge + 2

# Uneditable section #
def hf(electrons, num_qubits):
    """Calculates the Hartree-Fock state of the hydrogen molecule.
    
    Args:
        electrons (int): The number of electrons in the hydrogen molecule.
        num_qubits (int): The number of qubits needed to represent the hydrogen molecule Hamiltonian.

    Returns:
        (numpy.tensor): The HF state.
    """
    # End of uneditable section #

    # Put your solution here #
    return qml.qchem.hf_state(electrons, num_qubits)

hf(2, 4)

array([1, 1, 0, 0])

we have a spin down electron in lowest energy level, a spin up electron in the lowest energy level. and 0 spin down and 0 spin up electron in the second energy level.

$\downarrow\uparrow$ $\downarrow\uparrow$

In [49]:
# Uneditable section #
def run_VQE(coordinates, charge):
    """Performs a VQE routine for the given hydrogen molecule.

    Args:
        coordinates (list(float)): Cartesian coordinates of each hydrogen molecule.
        charge (int): The electric charge given to the hydrogen molecule.:

    Returns:
        (float): The expectation value of the hydrogen Hamiltonian.
    """

    hamiltonian = hydrogen_hamiltonian(np.array(coordinates), charge)

    electrons = num_electrons(charge)
    num_qubits = len(hamiltonian.wires)

    hf_state = hf(electrons, num_qubits)
    # singles and doubles are used to make the AllSinglesDoubles template
    # single and double excitations
    singles, doubles = qml.qchem.excitations(electrons, num_qubits)

    dev = qml.device("default.qubit", wires=num_qubits)

    @qml.qnode(dev)
    def cost(weights):
        """A circuit with tunable parameters/weights that measures the expectation value of the hydrogen Hamiltonian.
        
        Args:
            weights (numpy.array): An array of tunable parameters.

        Returns:
            (float): The expectation value of the hydrogen Hamiltonian.
        """
        # End of uneditable section #

        # Put your solution here #
        # AllSinglesDoubles -> apply the single and double excitations template
        qml.AllSinglesDoubles(weights, wires=range(num_qubits), hf_state=hf_state, singles=singles, doubles=doubles)
        return qml.expval(hamiltonian)

# Uneditable section #

    np.random.seed = 1234
    weights = np.random.normal(
        0, np.pi, len(singles) + len(doubles), requires_grad=True
    )
    opt = qml.AdamOptimizer(0.5)

    for _ in range(200):
        weights = opt.step(cost, weights)
        
    return cost(weights)

These functions are responsible for testing the solution. You will need to run the cell below. ***Do not modify the cell.***

In [50]:
def run(test_case_input: str) -> str:
    coordinates, charge = json.loads(test_case_input)
    energy = run_VQE(coordinates, charge)

    return str(energy)

def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(solution_output, expected_output, rtol=1e-3)

### Test cases
 Running the cell below will load the test cases. ***Do not modify the cell***.
 - input: [[0.0, 0.0, -0.8, 0.0, 0.0, 0.8], -1]
 	+ expected output: -0.53168359
 - input: [[0.0, 0.0, -0.6614, 0.0, 0.0, 0.6614], 0]
 	+ expected output: -1.13618883
 - input: [[0.0, 0.0, -0.72, 0.0, 0.0, 0.72], 1]
 	+ expected output: -0.54565673
 - input: [[0.0, 0.0, -0.45, 0.0, 0.0, 0.45], 0]
 	+ expected output: -1.03104296

In [51]:
test_cases = [['[[0.0, 0.0, -0.8, 0.0, 0.0, 0.8], -1]', '-0.53168359'], ['[[0.0, 0.0, -0.6614, 0.0, 0.0, 0.6614], 0]', '-1.13618883'], ['[[0.0, 0.0, -0.72, 0.0, 0.0, 0.72], 1]', '-0.54565673'], ['[[0.0, 0.0, -0.45, 0.0, 0.0, 0.45], 0]', '-1.03104296']]

### Solution testing
 Once you have run every cell above, including the one with your code, the cell below will test your solution. Run the cell. If you are correct for all of the test cases, it means your solutions is correct. Otherwise, you need to double check your work. ***Do not modify the cell below.***

In [52]:
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[[0.0, 0.0, -0.8, 0.0, 0.0, 0.8], -1]'...
Correct!
Running test case 1 with input '[[0.0, 0.0, -0.6614, 0.0, 0.0, 0.6614], 0]'...
Correct!
Running test case 2 with input '[[0.0, 0.0, -0.72, 0.0, 0.0, 0.72], 1]'...
Correct!
Running test case 3 with input '[[0.0, 0.0, -0.45, 0.0, 0.0, 0.45], 0]'...
Correct!
